# ROS 入门教程

## (一) ROS(robot operating system)简介

### (1.1) 架构和核心概念
1. 节点（Node）  
* 最小执行单元：每个节点是独立进程，负责特定功能（如传感器数据处理、运动控制）
* 分布式设计：节点可跨设备运行，通过 ROS Master 协调通信。
2. 通信机制  
* 主题（Topic）：节点通过主题传输数据（如传感器数据流），一个主题可被多个发布/订阅者使用。**消息**（Message）定义数据传输格式（如sensor_msgs/LaserScan表示激光雷达数据）。  
* 服务（Service）：**同步请求/响应模型** 用于需及时响应的操作（如开关控制），客户端请求后等待服务端响应。  
* 参数服务器（Parameter Server）：全局共享字典，用于存储配置参数（如机器人尺寸，算法阈值），节点可动态读写。  
3. 计算图（Computation Graph）：由节点、主题、服务等构成的网络，ROS Master 负责节点注册与通信协调。

### (1.2) 文件系统与工程管理
1. 功能包（Package）  
* 基本开发单元：包含节点源码、配置文件、依赖声明等。  
* 清单文件：包含包元数据及依赖关系。  
2. 工作空间（Workspace）  
* 使用 catkin 构建系统管理包，通过 catkin_make 编译代码。  
* 典型结构：src（存放源码）、build（存放编译的中间文件）、devel（存放生成的可执行文件）  
3. 堆栈。多个功能包的集合，提供完整的功能模块（如导航堆栈 navigation_stack）  

### (1.3) 开发工具和调试
1. 命令行工具
* rosnode：管理节点（启停、状态查询）。
* rostopic：监控主题消息（如 rostopic echo /scan）。
* rosservice：调用服务（如 rosservice call /set_goal）。
2. ​​可视化工具​​
* **​RViz**​​：3D 可视化环境，显示机器人模型、传感器数据及地图。
* ​rqt​​：图形化工具集（消息绘图、节点拓扑查看）。
* ​​Gazebo​​：物理仿真平台，支持传感器模拟与运动测试。
3. ​​调试与记录​​
* rosbag：记录和回放主题消息，用于算法离线验证

## (二) ROS2 安装和使用
中文文档地址：https://fishros.org/doc/ros2/humble/

### (2.1) ROS2 安装
我的操作系统是 ubuntu20.4，对应要安装的 ROS2 版本是 foxy，这里使用的是博主鱼香 ROS 的脚本日志分析，直接运行如下安装脚本，根据提示选择即可，建议安装桌面版。
```
wget http://fishros.com/install -O fishros && sudo bash fishros
```
安装完成后关闭所有终端，重新打开两个终端，在终端分别输入下面的命令，如果安装成功，那么终端 1 持续输出 Hello World，终端 2 同步接收显示。
```
ros2 run demo_nodes_cpp talker # 终端 1 运行发布节点
ros2 run demo_nodes_py listener # 终端 2 运行订阅节点
```
安装好后永久配置环境变量的方法如下：
```
echo "source /opt/ros/foxy/setup.bash" >> ~/.bashrc # 第一条指令
source ~/.bashrc # 第二条指令
```
用此方法安装的 ROS2 不能执行 `ros2 version` 命令，但是不影响使用。

### (2.2) 创建工作空间
创建一个名为 workSpace 的文件夹，在该文件夹内创建 src 和 build 文件夹
```
mkdir workSpace # 创建 workSpace
cd workSpace # 进入 workSpace
mkdir src build # 创建 src 和 build
catkin_make # 执行该命令构建系统管理包，会自动生成 devel 文件夹，用于放置可执行文件

# 包 Package 必须存放在 src 文件夹中，使用 catkin_create_pkg 命令创建
cd src
catkin_create_pkg hello_ros rospy rosmsg roscpp # 创建包 hello_ros ，并添加常用的三个依赖项 rospy（支持 python）、rosmsg（消息机制）、roscpp（支持 cpp）

# 回到 workSpace 文件夹，并在此打开终端，执行 catkin_make 进行编译，
```

### (2.3) Python 创建包和节点
#### (2.3.1) 面向过程创建节点
建议使用开发工具 vscode 创建工作空间、包和结点，如上创建工作空间之后，在 src 目录中创建包。  
首先打开终端，路径定位到 src 目录，因为所创建的包必须保存在 src 目录中，执行如下命令，其中 firstPkg 是包名，ament_python 是构建类型，rclpy 是依赖。  
```
os2 pkg create firstPkg --build-type ament_python --dependencies rclpy
```
然后在包的目录下有一个和包名一样的文件夹，这里就是 firstPkg，在这个文件夹下创建一个节点文件，这里创建 first.py ，内容如下：
```python
import rclpy
from rclpy.node import Node 

def main(args=None):
    rclpy.init(args=args) # 初始化
    hello_node=Node("hello") # 命名
    hello_node.get_logger().info("hello,world") # 输出日志信息
    rclpy.spin(hello_node) 
    rclpy.shutdown() # 停止
```

创建完节点文件之后，打开包中的 setup.py ，添加内容，才能在运行的时候找到节点，如下：
```python
entry_points={
        'console_scripts': [
            "first_node=firstPkg.first:main" # 这一行是添加的内容，first_node 是节点名，后面就是调用 firstPkg 文件夹中的 first.py 文件中的 main 函数。
        ],
    },
```
创建完 first.py 以及修改完 setup.py 之后，要执行编译，必须回到 src 的上一级目录，即工作空间中，执行编译命令 `colcon build` ，colcon 是编译器。再执行 `source install/setup.bash` 重新执行修改后的 setup.bash 文件，source（或点）命令通常用于重新执行刚修改的初始化文档，如 .bash_profile 和 .profile 这些配置文件。**这里由于直接双击 vscode 的快捷方式打开 vscode，所以可能不会加载环境变量，所以每次重新打开 vscode 都需要执行** `source install/setup.bash` ，比较麻烦，也有方法避免，自行百度。
然后就可以使用命令查找刚才的创建的包 firstPkg，比如 `ros2 pkg list | grep firstPkg` ，就会显示刚才创建的包 firstPkg。  
此时的节点可以运行，执行命令 `ros2 run firstPkg hello_node` 会输出日志信息，ros2 run 的格式是 `ros2 run 包名 节点名` ，节点名是在 setup.py 中指定的 hello_node ，并不是文件名 first 。

#### (2.3.2) 面向对象创建
使用类来实例化一个节点，在上面创建的 first.py 文件中添加节点，一个文件中可以创建多个节点，就是定义多个函数。
```python
import rclpy
from rclpy.node import Node

# 定义一个创建节点的类
class createNode(Node):
    def __init__(self,name):
        super().__init__(name)
        self.get_logger().info(f"hello ,i am {name} node")

# 定义 classNode 节点
def classNode(args=None):
    rclpy.init(args=args)
    classnode=createNode("miss") # 调用类来实例化节点
    rclpy.spin(classnode)
    rclpy.shutdown()
def main(args=None):
    rclpy.init(args=args)
    hello_node=Node("hello")
    hello_node.get_logger().info("hello,world")
    rclpy.spin(hello_node)
    rclpy.shutdown()
```
同章节 2.3.1 的步骤，节点创建完毕之后编辑节点所在包中的 setup.py 文件，添加节点 classnode，如下，节点之间要用 "," 隔开。
 ```python
entry_points={
        'console_scripts': [
            "first_node=firstPkg.first:main", # 这一行是添加的内容，first_node 是节点名，后面就是调用 firstPkg 文件夹中的 first.py 文件中的 main 函数。
            "classnode=firstPkg.first:classNode"
        ],
    },
```
最后回到工作空间所在目录，执行 `colcon build` 编译，执行 `source installsetup.bash` ，执行 `ros2 run firstPkg classnode` 即可运行节点 classnode。

### (2.4) cpp 创建包和节点
直接使用面向过程创建。进入工作空间中的 src 目录，因为包必须放在该目录，执行下面的命令创建包 cppPkg。创建类型是 ament_cmake ，可以不写，因为默认就是 ament_cmake，依赖是 rclcpp。
```
ros2 pkg create cppPkg --build-type ament_cmake --dependencies rclcpp
```
包创建完成后，在 cppPkg/src 文件夹中创建 cpp 文件 cppNode.cpp ，内容如下，vscode 可能识别不了头文件，自行百度解决，这个不影响编译，只是 vscode 的问题。
```c++
#include "rclcpp/rclcpp.hpp"

int main(int argc,char** argv)
{
    rclcpp::init(argc,argv); // 初始化
    auto node=std::make_shared<rclcpp::Node>("firstCPP"); // 创建节点
    RCLCPP_INFO(node->get_logger(),"hello,this is the first cpp node") // 打印日志
    rclcpp::spin(node);
    rclcpp::shutdown(); // 检测到 ctrl+c 退出
    return 0;
}
```
然后在包 cppPkg 目录下的 CMakeLists.txt 的最后添加以下内容，第一行是添加可执行文件，第二行是添加依赖，第三行安装节点。
```
add_executable(first_cpp_node src/cppNode.cpp)
ament_target_dependencies(first_cpp_node rclcpp)
install(TARGETS first_cpp_node DESTINATION lib/${PROJECT_NAME})
```
最后回到工作空间所在的目录，执行 `colcon build --packages-select cppPkg` 编译包 cppPkg，接着执行 `source install/setup.bash` ，执行 `ros2 pkg list | grep cppPkg` 查询包是否存在，`ros2 run cppPkg first_cpp_node` 执行节点 first_cpp_node 。